In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.runBlocking

val username = "Johann-25.0"

val intervalStart = LocalDate.parse("2019-01-01")
val intervalEnd = LocalDate.parse("2021-01-31")

val fundName = "Work Income"
val reportViewName = "Work income report X"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/work-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = runBlocking { client.ensureUserExists(username) }
user

UserTO(id=2811ee93-fb16-43aa-9892-60058eec39b7, username=Johann-25.0)

In [3]:
val reportView = runBlocking {
    client.createReportView(user, reportViewName, fundName, File(REPORT_DATA_CONFIGURATION_YAML_FILE))
}
reportView

ReportViewTO(id=e8ac515c-da03-41a8-b8a6-939636659368, name=Work income report X, fundId=65cd5c32-0b6f-4c08-b256-ae27aefa5807, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(enabled=false, inputBuckets=0, outputBuckets=0))))

In [4]:
val monthlyReportData = runBlocking {
    client.getReportViewData(user, reportViewName, intervalStart, intervalEnd, TimeGranularity.MONTHLY)
}

In [8]:
val yearlyReportData = runBlocking {
    client.getReportViewData(user, reportViewName, intervalStart, intervalEnd, TimeGranularity.YEARLY)
}

In [6]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupedNetData(reportData: ReportDataTO, plotTitle: String, groupName: String): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "net" to reportData.data.map { bucket ->
            bucket.groupedNet!!.first { it.group == groupName }.net
        },
    )

    return dataFrame
        .plot {
            line {
                val values = dataFrame.get("net").values.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.granularInterval.granularity) {
                    TimeGranularity.YEARLY -> intervalEnd.minus(183, DateTimeUnit.DAY)
                    else -> intervalEnd.minus(15, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.granularInterval.granularity) {
                    TimeGranularity.YEARLY -> "%Y"
                    else -> "%b %Y"
                }
                axis.breaks(reportData.data.map { it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds() }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }

}

In [10]:
plotGroupedNetData(yearlyReportData, "Net income per year", "income")
// TODO(Johann) yearly report should contain only fixed years for more accurate forecast. or could forecasting extrapolate the rest of the year?

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="10kqNM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Net income per year"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"net":[107567.0,138677.75,13596.0,21653.39583333333,23457.84548611111,25412.66594328704,27530.38810522762,29824.58711399659,32309.96937349631]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[0.0,138677.75]
},
"sampling":"none",
"x":1.59624E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"5"
};
 var containerDiv = document.getElementById("10kqNM");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 2023 
 
 
 
 
 
 
 
 
 2024 
 
 
 
 
 
 
 
 
 2025 
 
 
 
 
 
 
 
 
 2026 
 
 
 
 
 
 
 
 
 2027 
 
 
 
 
 
 
 
 
 
 
 -5,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 25,000 
 
 
 
 
 
 
 30,000 
 
 
 
 
 
 
 35,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 45,000 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 55,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 65,000 
 
 
 
 
 
 
 70,000 
 
 
 
 
 
 
 75,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 85,000 
 
 
 
 
 
 
 90,000 
 
 
 
 
 
 
 95,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 105,000 
 
 
 
 
 
 
 110,000 
 
 
 
 
 
 
 115,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 125,000 
 
 
 
 
 
 
 130,000 
 
 
 
 
 
 
 135,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 145,000 
 
 
 
 
 
 
 
 
 Net income per year 
 
 
 
 
 net 
 
 
 
 
 bucket

In [7]:
plotGroupedNetData(monthlyReportData, "Income per month", "income")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="o78xNa"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Income per month"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"net":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,11841.47916666667,11975.18576388889,12120.70124421296,11516.84301456404,11525.66326577771,11649.46853792586]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[0.0,19367.0]
},
"sampling":"none",
"x":1.6107552E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("o78xNa");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M105.13743101878936 1055.1545454545455 L105.13743101878936 1055.1545454545455 L176.6124389482211 1055.1545454545455 L241.17051062642713 1055.1545454545455 L312.64551855586615 1055.1545454545455 L381.8148810682251 1055.1545454545455 L453.2898889976641 1055.1545454545455 L522.459251510023 1055.1